In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  read the dataset
df = pd.read_csv("2012-sat-results.csv")

print(df.info())
print("")

# convert all values to numeric
df["SAT Critical Reading Avg. Score"] = pd.to_numeric(df["SAT Critical Reading Avg. Score"], errors="coerce")
df["SAT Math Avg. Score"] = pd.to_numeric(df["SAT Math Avg. Score"], errors="coerce")
df["SAT Writing Avg. Score"] = pd.to_numeric(df["SAT Writing Avg. Score"], errors="coerce")
# Drop rows with NaN values
df = df.dropna(subset=["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"])

print(df.info())
print("")

# population params
mu = df["SAT Writing Avg. Score"].mean()
tao = df["SAT Writing Avg. Score"].sum()
sigmasq = df["SAT Writing Avg. Score"].var(ddof=0)

print(f"The mu is: {mu}")
print(f"The tao is: {tao}")
print(f"The sigma^2 is: {sigmasq}")

print("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              421 non-null    object 
 1   SCHOOL NAME                      421 non-null    object 
 2   Num of SA

In [2]:
df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0


# 1. Divide your population into strata

We experimented with many different stratum binning sizes. However, we were not able to find bin sizes that were able to minimize the variance
in every stratum. This caused the data to be skewed, and result in mu not being within the 95% CI's calculated. We believe this is an issue with the
data.

In [5]:

# Creating Strata based on Number of SAT Test Takers per school

df["Num of SAT Test Takers"] = pd.to_numeric(df["Num of SAT Test Takers"], errors='coerce')

# strata ranges
bins = [0, 50, 100, 200, 400, df["Num of SAT Test Takers"].max()]
labels = [0, 1, 2, 3, 4]

df["Testers_Stratum"] = pd.cut(df["Num of SAT Test Takers"], bins=bins, labels=labels, include_lowest=True)

testers = {
    "Nh": df.groupby("Testers_Stratum", observed=False).size().tolist(),
    "sigma_sq_h": df.groupby("Testers_Stratum", observed=False)["SAT Writing Avg. Score"].var(ddof=1).tolist()
}
testers

{'Nh': [148, 173, 50, 26, 24],
 'sigma_sq_h': [979.5716124287546,
  2186.963099879016,
  6055.375102040816,
  5672.91846153846,
  7052.606884057976]}

In [6]:
# Creating Strata based on SAT Critical Reading Avg. Score

# Intervals for stratum and labels for the intervals
bins = [280, 340, 370, 400, 430, 460, 500, 550, df["SAT Critical Reading Avg. Score"].max()]
labels = list(range(len(bins) - 1))

df["Reading_Stratum"] = pd.cut(df["SAT Critical Reading Avg. Score"], bins=bins, labels=labels, include_lowest=True)

# Grouping by stratum, and calculating variances for each stratum
reading = {
    "Nh": df.groupby("Reading_Stratum", observed=False).size().tolist(),
    "sigma_sq_h": df.groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"].var(ddof=1).tolist()
}
reading

{'Nh': [26, 94, 137, 86, 30, 23, 11, 13],
 'sigma_sq_h': [326.7215384615385,
  185.81972088766875,
  207.09360240446514,
  292.74815321477445,
  308.5241379310345,
  465.086956521739,
  370.65454545454566,
  1131.8076923076926]}

In [7]:
# Creating Strata based on SAT Math Avg. Score

# Intervals for stratum and labels for the intervals
bins = [280, 340, 370, 400, 430, 460, 500, 550, df["SAT Critical Reading Avg. Score"].max()]
labels = list(range(len(bins) - 1))

df["Math_Stratum"] = pd.cut(df["SAT Math Avg. Score"], bins=bins, labels=labels, include_lowest=True)

# Grouping by stratum, and calculating variances for each stratum
math = {
    "Nh": df.groupby("Math_Stratum", observed=False).size().tolist(),
    "sigma_sq_h": df.groupby("Math_Stratum", observed=False)["SAT Writing Avg. Score"].var(ddof=1).tolist()
}
math

{'Nh': [15, 83, 133, 73, 45, 38, 12, 19],
 'sigma_sq_h': [625.6000000000003,
  404.5433441081398,
  364.5635680109365,
  441.43759512937663,
  498.1545454545457,
  1761.76386913229,
  3116.265151515152,
  5642.005847953215]}

# 2. Evaluate Population Stratification

In [9]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(testers["Nh"])
sigma_sq_h = np.array(testers["sigma_sq_h"])

delta_testers = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_testers

319661.0248756949

In [10]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(reading["Nh"])
sigma_sq_h = np.array(reading["sigma_sq_h"])

delta_reading = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_reading

1325597.0586936367

In [11]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(math["Nh"])
sigma_sq_h = np.array(math["sigma_sq_h"])

delta_math = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_math

1095786.067822362

In [12]:
max_delta =  max( max(delta_math, delta_reading), delta_testers )
print(f"Largest delta {max_delta}")

Largest delta 1325597.0586936367


#### Use SAT Critical Reading Avg. Score strata from here on

# Sampling Procedures

### 3. Stratified Random Sample with Equal Allocation:
Take stratified random sample with size n (chosen in Report 2) with equal allocation. $n_h = \frac{n}{L}$

In [15]:
# use reading strata divisions
reading

{'Nh': [26, 94, 137, 86, 30, 23, 11, 13],
 'sigma_sq_h': [326.7215384615385,
  185.81972088766875,
  207.09360240446514,
  292.74815321477445,
  308.5241379310345,
  465.086956521739,
  370.65454545454566,
  1131.8076923076926]}

In [16]:
# n chosen in report 2 is 80.
N_h = np.array(reading['Nh'])
sigma_sq_h = np.array(reading['sigma_sq_h'])

n = 80
L = len(reading["Nh"])
n_h = int(np.ceil(n / L))
n_h

10

In [17]:
stratified_samples = []

for stratum_label in range(L):
    stratum = df[df["Reading_Stratum"] == stratum_label]
    stratified_samples.append(stratum.sample(n=n_h, replace=False, random_state=420))
stratified_sample_df = pd.concat(stratified_samples).reset_index(drop=True)
stratified_sample_df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Testers_Stratum,Reading_Stratum,Math_Stratum
0,18K563,IT TAKES A VILLAGE ACADEMY,56,313.0,320.0,330.0,1,0,0
1,10X268,KINGSBRIDGE INTERNATIONAL HIGH SCHOOL,52,304.0,356.0,302.0,1,0,1
2,12X388,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONROE,30,321.0,351.0,298.0,0,0,1
3,02M394,EMMA LAZARUS HIGH SCHOOL,79,319.0,512.0,357.0,1,0,6
4,02M542,MANHATTAN BRIDGES HIGH SCHOOL,66,336.0,378.0,344.0,1,0,2
...,...,...,...,...,...,...,...,...,...
75,03M485,FIORELLO H. LAGUARDIA HIGH SCHOOL OF MUSIC & A...,531,566.0,564.0,577.0,4,7,7
76,10X696,HIGH SCHOOL OF AMERICAN STUDIES AT LEHMAN COLLEGE,92,636.0,648.0,636.0,1,7,7
77,25Q525,TOWNSEND HARRIS HIGH SCHOOL,278,621.0,651.0,638.0,3,7,7
78,28Q687,QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK CO...,121,612.0,660.0,596.0,2,7,7


# 4. Parameter Estimation and Variance

### a) Estimate your parameter of interest using an unbiased estimator
$\hat{\tau_{\text{st}}} = \sum_{h=1}^L N_h\bar{y}_h$

$\hat{\mu_{\text{st}}} = \frac{1}{N}\hat{\tau_{\text{st}}}$

In [19]:
# mu_st_hat = tau_st_hat / N
# tau_st_hat = sum from 1 to L of (N_h * ybar_h)
# ybar_h = (sum from 1 to n_h of (y_hi)) / n_h

ybar_h = stratified_sample_df.groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"].mean().tolist()
tau_st_hat = sum(N_h * ybar_h)
mu_st_hat = tau_st_hat / N
round(float(mu_st_hat), 2)

396.49

### b) Estimate its variance and provide a confidence interval at the alpha level chosen in Report 2.
$\hat{\text{var}}(\hat{\tau_{\text{st}}}) = \sum_{h=1}^L N_h(N_h - n_h)\frac{s^2_h}{n_h}$

$\hat{\text{var}}(\hat{\mu_{\text{st}}}) = \frac{1}{N^2}\hat{\text{var}}(\hat{\tau_{\text{st}}})$

In [21]:
# variance
# var_hat_mu_st_hat = (1/N^2) * var_hat_tau_st_hat
# var_hat_tau_st_hat = sum from 1 to L ((N_h * (N_h - n_h) * (sigma_h^2 / n_h)))
s_h_squared = (
    stratified_sample_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s_h_squared = np.array(s_h_squared)
var_hat_tau_st_hat = sum(N_h * (N_h - n_h) * (s_h_squared / n_h))
var_hat_mu_st_hat = (1 / N**2) * var_hat_tau_st_hat
round(float(var_hat_mu_st_hat), 2)

3.44

### c) Use the Satterthwaite formula for adjusted degrees of freedom
$\large a_h = \frac{N_h(N_h - n_h)}{n_h}$

$\large d = \frac{(\sum_{h=1}^L a_h s^2_h)^2}{\sum_{h=1}^L \frac{(a_h s^2_h)^2}{n_h - 1}}$

In [23]:
# adjusted degrees of freedom
# d = (sum of 1 to L of (a_h * s_h^2))
#      / (sum of 1 to L of (a_h * s_h^2) / (n_h - 1))
# a_h = (N_h * (N_h - n_h)) / n_h
s_h_squared = (
    stratified_sample_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s_h_squared = np.array(s_h_squared)
a_h = N_h * (N_h - n_h) / n_h
d = sum(a_h * s_h_squared)**2 / sum((a_h * s_h_squared)**2 / (n_h - 1))
float(d)

27.335148972027742

In [24]:
from scipy.stats import t
# Confidence Interval with alpha = .05
# mu_st_hat +- t(alpha/2, d) * sqrt(var_hat_mu_st_hat)
alpha = .05
t_crit = t.ppf(1-(alpha/2), d)
SDE = t_crit * np.sqrt(var_hat_mu_st_hat)

CI = mu_st_hat - SDE, mu_st_hat + SDE
print(f"95% CI for mu with Stratified Random Sample with Equal Allocation is: ({CI[0].round(2)}, {CI[1].round(2)}) \n")

95% CI for mu with Stratified Random Sample with Equal Allocation is: (392.69, 400.3) 



# 5. Stratified Random Sample with Proportional Allocation

$n_h = \frac{nN_h}{N}$

In [26]:
nh_proportional = [
    int(round((n * Nh) / N))
    for Nh in reading["Nh"]
]
nh_proportional

[5, 18, 26, 16, 6, 4, 2, 2]

In [27]:
while sum(nh_proportional) != n:
    if sum(nh_proportional) < n:
        nh_proportional[nh_proportional.index(min(nh_proportional))] += 1
    else:
        nh_proportional[nh_proportional.index(max(nh_proportional))] -= 1

stratified_samples_prop = []
for stratum_label in range(L):
    stratum_data = df[df["Reading_Stratum"] == stratum_label]
    sample_size = nh_proportional[stratum_label]

    if sample_size > 0:
        sample = stratum_data.sample(n=sample_size, replace=False)
        stratified_samples_prop.append(sample)
stratified_sample_prop_df = pd.concat(stratified_samples_prop).reset_index(drop=True)

stratified_sample_prop_df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Testers_Stratum,Reading_Stratum,Math_Stratum
0,24Q296,PAN AMERICAN INTERNATIONAL HIGH SCHOOL,55,317.0,323.0,311.0,1,0,0
1,09X365,ACADEMY FOR LANGUAGE AND TECHNOLOGY,54,315.0,339.0,297.0,1,0,0
2,09X227,BRONX EXPEDITIONARY LEARNING HIGH SCHOOL,39,324.0,376.0,349.0,0,0,2
3,20K658,FRANKLIN DELANO ROOSEVELT YABC,18,338.0,477.0,316.0,0,0,5
4,12X550,HIGH SCHOOL OF WORLD CULTURES,42,304.0,323.0,312.0,0,0,0
...,...,...,...,...,...,...,...,...,...
75,02M412,N.Y.C. LAB SCHOOL FOR COLLABORATIVE STUDIES,114,537.0,590.0,550.0,2,6,7
76,02M413,SCHOOL OF THE FUTURE HIGH SCHOOL,66,517.0,533.0,515.0,1,6,6
77,02M418,MILLENNIUM HIGH SCHOOL,144,528.0,553.0,533.0,2,6,7
78,05M692,"HIGH SCHOOL FOR MATHEMATICS, SCIENCE AND ENGIN...",101,605.0,654.0,588.0,2,7,7


 ## 6. Parameter Estimation and Variance

### a) Estimate your parameter of interest using an unbiased estimator
$\hat{\tau_{\text{st}}} = \sum_{h=1}^L N_h\bar{y}_h$

$\hat{\mu_{\text{st}}} = \frac{1}{N}\hat{\tau_{\text{st}}}$

In [30]:
ybar_h_prop= stratified_sample_prop_df.groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"].mean().tolist()

tau_st_hat_prop=sum(N_h * ybar_h_prop)
mu_st_hat_prop = tau_st_hat_prop/N

round(float(mu_st_hat_prop), 2)

394.25

### b) Estimate its variance and provide a confidence interval at the α level chosen in Report 2.
$\hat{\text{var}}(\hat{\tau_{\text{st}}}) = \sum_{h=1}^L N_h(N_h - n_h)\frac{s^2_h}{n_h}$

$\hat{\text{var}}(\hat{\mu_{\text{st}}}) = \frac{1}{N^2}\hat{\text{var}}(\hat{\tau_{\text{st}}})$

In [32]:
s_h_squared_prop = (
    stratified_sample_prop_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s_h_squared_prop = np.array(s_h_squared_prop)

var_hat_tau_st_hat_prop = sum(N_h * (N_h - nh_proportional) * (s_h_squared_prop / nh_proportional))
var_hat_mu_st_hat_prop = (1/(N**2)) * var_hat_tau_st_hat_prop

round(float(var_hat_mu_st_hat_prop), 2)

1.93

### c) Use the Satterthwaite formula for adjusted degrees of freedom.
$\large a_h = \frac{N_h(N_h - n_h)}{n_h}$

$\large d = \frac{(\sum_{h=1}^L a_h s^2_h)^2}{\sum_{h=1}^L \frac{(a_h s^2_h)^2}{n_h - 1}}$

In [34]:
s_h_squared_prop = (
    stratified_sample_prop_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s_h_squared_prop = np.array(s_h_squared_prop)
nh_proportional = np.array(nh_proportional)
a_h = N_h * (N_h - nh_proportional) / nh_proportional
d_prop = sum(a_h * s_h_squared_prop)**2 / sum((a_h * s_h_squared_prop)**2 / (nh_proportional - 1))
float(d_prop)

55.89778175914819

In [35]:
from scipy.stats import t

alpha = 0.05
t_crit = t.ppf(1-(alpha/2), d_prop)

SDE = t_crit*np.sqrt(var_hat_mu_st_hat_prop)
CI_prop = mu_st_hat_prop - SDE, mu_st_hat_prop + SDE
print(f"95% CI for mu with Stratified Random Sample with Proportional Allocation is: ({CI_prop[0].round(2)}, {CI_prop[1].round(2)}) \n")

95% CI for mu with Stratified Random Sample with Proportional Allocation is: (391.47, 397.03) 



# 7. Stratified Random Sample with Optimum Allocation

$n_h = \frac{nN_h\sigma_h}{\sum_{h=1}^L N_h\sigma_h}$

In [37]:
N_h = np.array(reading['Nh'])
sigma2_h = np.array(reading['sigma_sq_h'])
sigma_h = np.sqrt(sigma2_h)

N = len(df)
L = len(N_h)
n = 80

n_h_opt = ((n * N_h * sigma_h) / sum(N_h * sigma_h)).round()
n_h_opt = n_h_opt.astype(int).tolist()
n_h_opt

[5, 15, 23, 17, 6, 6, 2, 5]

In [38]:
stratified_samples_opt = []
for strata in range(L):
    stratum_data = df[df["Reading_Stratum"] == strata]
    n = n_h_opt[strata]

    if n > 0:
        sample = stratum_data.sample(n=n, replace=False, random_state=420)
        stratified_samples_opt.append(sample)

stratified_sample_opt_df = pd.concat(stratified_samples_opt).reset_index(drop=True)
stratified_sample_opt_df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Testers_Stratum,Reading_Stratum,Math_Stratum
0,18K563,IT TAKES A VILLAGE ACADEMY,56,313.0,320.0,330.0,1,0,0
1,10X268,KINGSBRIDGE INTERNATIONAL HIGH SCHOOL,52,304.0,356.0,302.0,1,0,1
2,12X388,PAN AMERICAN INTERNATIONAL HIGH SCHOOL AT MONROE,30,321.0,351.0,298.0,0,0,1
3,02M394,EMMA LAZARUS HIGH SCHOOL,79,319.0,512.0,357.0,1,0,6
4,02M542,MANHATTAN BRIDGES HIGH SCHOOL,66,336.0,378.0,344.0,1,0,2
...,...,...,...,...,...,...,...,...,...
74,02M475,STUYVESANT HIGH SCHOOL,832,679.0,735.0,682.0,4,7,NaN
75,31R605,STATEN ISLAND TECHNICAL HIGH SCHOOL,227,635.0,682.0,636.0,3,7,NaN
76,05M692,"HIGH SCHOOL FOR MATHEMATICS, SCIENCE AND ENGIN...",101,605.0,654.0,588.0,2,7,7
77,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624.0,604.0,628.0,2,7,7


### 7a) Parameter Estimation and Variance

### b) Estimate your parameter of interest using an unbiased estimator
$\hat{\tau_{\text{st}}} = \sum_{h=1}^L N_h\bar{y}_h$

$\hat{\mu_{\text{st}}} = \frac{1}{N}\hat{\tau_{\text{st}}}$

In [41]:
y_bar_h_opt = stratified_sample_opt_df.groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"].mean().tolist()

tau_st_hat_opt = sum(N_h * y_bar_h_opt)
mu_st_hat_opt = tau_st_hat_opt / N

round(float(mu_st_hat_opt), 2)

393.97

### c) Estimate its variance and provide a confidence interval at the $\alpha$ level chosen in Report 2.
$\hat{\text{var}}(\hat{\tau_{\text{st}}}) = \sum_{h=1}^L N_h(N_h - n_h)\frac{s^2_h}{n_h}$

$\hat{\text{var}}(\hat{\mu_{\text{st}}}) = \frac{1}{N^2}\hat{\text{var}}(\hat{\tau_{\text{st}}})$

In [43]:
s2_h_opt = (
    stratified_sample_opt_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s2_h_opt = np.array(s2_h_opt)
var_hat_tau_st_hat_opt = sum(N_h * (N_h - n_h_opt) * (s2_h_opt / n_h_opt))

var_hat_mu_st_hat_opt = (1 / (N**2)) * var_hat_tau_st_hat_opt

round(float(var_hat_mu_st_hat_opt), 2)

2.66

### d) Use the Satterthwaite formula for adjusted degrees of freedom
$\large a_h = \frac{N_h(N_h - n_h)}{n_h}$

$\large d = \frac{(\sum_{h=1}^L a_h s^2_h)^2}{\sum_{h=1}^L \frac{(a_h s^2_h)^2}{n_h - 1}}$

In [45]:
s2_h_opt = (
    stratified_sample_opt_df
    .groupby("Reading_Stratum", observed=False)["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
s2_h_opt = np.array(s2_h_opt)
n_h_opt = np.array(n_h_opt)

a_h_opt = N_h * (N_h - n_h_opt) / n_h_opt

d_opt = sum(a_h * s2_h_opt)**2 / sum((a_h * s2_h_opt)**2 / (n_h_opt - 1))
float(d_opt)

37.83128412377717

In [46]:
from scipy.stats import t

alpha = 0.05
t_crit = t.ppf(1-alpha/2, d_opt)

SDE = t_crit * float(np.sqrt(var_hat_mu_st_hat_opt))

CI_opt = mu_st_hat_opt - SDE, mu_st_hat_opt + SDE
print(f"95% CI for mu with Stratified Random Sample with Optimum Allocation is: ({CI_opt[0].round(2)}, {CI_opt[1].round(2)}) \n")

95% CI for mu with Stratified Random Sample with Optimum Allocation is: (390.67, 397.27) 



# 8. Best Estimator Selection

In [48]:
print(CI[0].round(2).astype(float), CI[1].round(2).astype(float))
print(CI_prop[0].round(2).astype(float), CI_prop[1].round(2).astype(float))
print(CI_opt[0].round(2).astype(float), CI_opt[1].round(2).astype(float))
print()
print("Variance of mu estimate for Equal allocation:", round(var_hat_mu_st_hat, 3))
print("Variance of mu estimate for Proportional allocation:", round(var_hat_mu_st_hat_prop, 3))
print("Variance of mu estimate for Optimal allocation:", round(var_hat_mu_st_hat_opt, 3))

392.69 400.3
391.47 397.03
390.67 397.27

Variance of mu estimate for Equal allocation: 3.444
Variance of mu estimate for Proportional allocation: 1.927
Variance of mu estimate for Optimal allocation: 2.657


Of the three estimators, the estimator for $\mu$ under Proportional Allocation has the lowest variance of $1.927$. Therefore it is the best Estimator of the three.